In [6]:
import pandas as pd

activities = pd.read_csv("./../../data/activitats.csv", encoding='ISO-8859-1')
marks = pd.read_csv("./../../data/notes.csv", delimiter=';')
submision = pd.read_csv("./../../data/trameses.csv")


Academic Record

In [15]:
user_id = 155
marks_user = marks[(marks["userid"] == user_id)]
marks_user_record = marks_user[["aula_id"]]


In [11]:
def Academic_Record(user_id):
    return marks[(marks["userid"] == user_id)]

In [12]:
Academic_Record(155)

,userid,aula_id,P_Grade,P_Grade_Date,F_Grade,F_Grade_Date,R_Grade,R_Grade_Date
34,155,92,NaN,NaN,"5,75",1667829600,NaN,1675173600
352,155,143,NaN,NaN,"8,65",1699884000,NaN,1706191200
